In [3]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

from IPython.display import Audio

### <font color=blue> Prep data. Choose deaths transformation.

In [4]:
from epsampling.modeling import get_full_modeling_dfs

df, df_acs = get_full_modeling_dfs(timestamp='20241009-144131')

# df = df[df.True_inc_deaths > 0]

# df['Target'] = df.apply(lambda x: (x.True_inc_deaths+1/100000) / x.Pop * 100000, axis=1)
# df['Naive'] = df.apply(lambda x: (x.Naive_inc_deaths+1/100000) / x.Pop * 100000, axis=1)
# df['State'] = df.apply(lambda x: (x.Proj_inc_deaths+1/100000) / x.State_pop * 100000, axis=1)
# df['Cumtm1'] = df.apply(lambda x: (x.Cum_deaths_tm1+1/100000) / x.Pop * 100000, axis=1)

df['Target'] = df.apply(lambda x: (x.True_inc_deaths+1) / x.Pop * 100000, axis=1)
df['Naive'] = df.apply(lambda x: (x.Naive_inc_deaths+1) / x.Pop * 100000, axis=1)
df['State'] = df.apply(lambda x: (x.Proj_inc_deaths+1) / x.State_pop * 100000, axis=1)
df['Cumtm1'] = df.apply(lambda x: (x.Cum_deaths_tm1+1) / x.Pop * 100000, axis=1)

# df['Target'] = df.apply(lambda x: (x.True_inc_deaths / x.Pop + 1) * 100000, axis=1)
# df['Naive'] = df.apply(lambda x: (x.Naive_inc_deaths / x.Pop + 1) * 100000, axis=1)
# df['State'] = df.apply(lambda x: (x.Proj_inc_deaths / x.State_pop + 1) * 100000, axis=1)
# df['Cumtm1'] = df.apply(lambda x: (x.Cum_deaths_tm1 / x.Pop + 1) * 100000, axis=1)

# df['Target'] = df.apply(lambda x: x.True_inc_deaths, axis=1)
# df['Naive'] = df.apply(lambda x: x.Naive_inc_deaths, axis=1)
# df['State'] = df.apply(lambda x: x.Proj_inc_deaths, axis=1)
# df['Cumtm1'] = df.apply(lambda x: x.Cum_deaths_tm1, axis=1)

# df['Target'] = df.apply(lambda x: x.True_inc_deaths + 1, axis=1)
# df['Naive'] = df.apply(lambda x: x.Naive_inc_deaths + 1, axis=1)
# df['State'] = df.apply(lambda x: x.Proj_inc_deaths + 1, axis=1)
# df['Cumtm1'] = df.apply(lambda x: x.Cum_deaths_tm1 + 1, axis=1)

# df['Target'].describe()

### <font color=blue> Do subset where true deaths is more than 0?

In [15]:
# df = df[df.True_inc_deaths > 0]

# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# df['Target'].describe(), df['Naive'].describe()

In [ ]:
### <font color=blue> Run models.

In [25]:
from epsampling.utils import get_chunks
from epsampling.modeling import get_date_chunked_splits, get_performance
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Lasso
random.seed(666)

# # # # # # # # # #
num_runs = 4
alg = 'lasso'
# # # # # # # # # #

chunks = get_chunks(list(df.Date.unique()), 4)
idc = random.sample(range(len(chunks)), 4, )
idc = [14,12]

metrics_dicts = {}


for idx in idc:
    
    df_train, df_test = get_date_chunked_splits(df, chunks, idx, samp_frac=0.5)
    
    ## WHICH FEATS ??? #######

    feats = list(df_acs.columns) + ['State','Cumtm1','Naive',
                                   'State_pop','Ratio','Pop']
    
#     feats = ['POP_NHPINH', 'POP_OTH2PLNH', 'POP_ASIANNH', 'POP_AIANNH',
#              'POP_HISP_x12', 'POP_NH', 'POP_BLACKNH', 'POP_WHITENH','IND_WHT_x7', 
#              'EDU_BACH', 'HHI_150_200k'] + ['State','Cumtm1','Naive','State_pop','Ratio','Pop']
    
#     feats = ['State','Cumtm1','Naive','State_pop','Ratio','Pop']
    
#     feats = ['POP_NHPINH', 'POP_OTH2PLNH', 'POP_ASIANNH', 'POP_AIANNH',
#              'POP_HISP_x12', 'POP_NH', 'POP_BLACKNH', 'POP_WHITENH','IND_WHT_x7', 
#              'EDU_BACH', 'HHI_150_200k'] +  ['State','Cumtm1','Naive',
#                                    'State_pop','Ratio','Pop']
    
#     feats = ['POP_NHPINH', 'POP_OTH2PLNH', 'POP_ASIANNH', 'POP_AIANNH',
#              'POP_HISP_x12', 'POP_NH', 'POP_BLACKNH', 'POP_WHITENH','IND_WHT_x7', 
#              'EDU_BACH', 'HHI_150_200k'] 
        
    X_train = df_train[feats]
    X_test = df_test[feats]

    y_train = df_train['Target']
    y_test = df_test['Target']
    y_naive = df_test['Naive']
    
    ## IS TARGET DATA WEIRD? #########
#     print(y_train.max(), y_train.min(), y_test.max(), y_test.min())
    
    if alg=='linear':
        reg = LinearRegression().fit(X_train, y_train)
    elif alg=='lasso':
        reg = Lasso(alpha=0.25).fit(X_train, y_train)

    
    y_pred = reg.predict(X_test)
    y_naive = df_test['Naive']

    model_names = ['Deaths_pred', 'Deaths_naive']
    model_preds = [y_pred, y_naive]

    metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
    metrics_dicts[idx] = metrics_dict

    
bad_runs = []
good_runs = []

for i,metrics_dict in metrics_dicts.items():
    print(f'\n\n====== RUN {i} ======')
    for k,v in metrics_dict.items():
        print(f' > {k}')
        for k2,v2 in v.items():
            
            if v2 > 100000000:
                bad_runs.append(i)
            if k2=='r2':
                k3 = 'r-squared'
            else:
                k3 = k2
            print(f'  • {k3}: \t{round(v2,5)}')
            
bad_models = [metrics_dicts[i] for i in bad_runs]
good_models = [metrics_dicts[i] for i in idc if i not in bad_runs]



====== RUN 14 ======
 > Deaths_pred
  • MAE: 	2.36568
  • MedAE: 	1.63533
  • MSE: 	22.49911
  • RMSE: 	4.74332
  • r-squared: 	0.99308
  • relMAE: 	2.5173
  • relMedAE: 	1.83207
 > Deaths_naive
  • MAE: 	1.86127
  • MedAE: 	0.8142
  • MSE: 	22.223
  • RMSE: 	4.71413
  • r-squared: 	0.99316
  • relMAE: 	1.0
  • relMedAE: 	1.0


====== RUN 12 ======
 > Deaths_pred
  • MAE: 	1.86097
  • MedAE: 	1.60145
  • MSE: 	8.5299
  • RMSE: 	2.9206
  • r-squared: 	0.99597
  • relMAE: 	4650310951531.943
  • relMedAE: 	2.15325
 > Deaths_naive
  • MAE: 	1.27961
  • MedAE: 	0.69367
  • MSE: 	7.95264
  • RMSE: 	2.82004
  • r-squared: 	0.99625
  • relMAE: 	0.99657
  • relMedAE: 	1.0


### <font color=blue> Plots.

In [27]:
# # # # # # # # # # #
# how_many = 8
# # # # # # # # # # #

# coef = abs(reg.coef_)
# ind = np.argpartition(coef, -how_many)[-how_many:]
# top = coef[ind]

# df_coefs = X_train.iloc[:, ind]
# which_feats = df_coefs.columns

# res_df = pd.DataFrame([top, which_feats], index=['coef','feat_name'])
# res_df = res_df.T.sort_values(by='coef',inplace=False,ascending=False)
# res_df = res_df[:how_many]

# import seaborn as sns
# import matplotlib.pyplot as plt


# DPI = 300

# plt.figure(figsize=(10,5))
# sns.barplot(res_df, y="coef", x="feat_name", legend=False)
# plt.title(f'{alg}')
# # plt.xticks(xticks, rotation=45)
# plt.xticks(rotation=45,ha='right',wrap='True')

# sns.despine()
# plt.gcf().set_dpi(DPI)
# plt.show()